In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

In [67]:
import geopandas as gpd
import pandas as pd
from siuba import *
import numpy as np

from segment_speed_utils import helpers, gtfs_schedule_wrangling
from shared_utils import rt_dates, gtfs_utils_v2
import folium
import itertools

In [3]:
from update_vars import (analysis_date, AM_PEAK, PM_PEAK, EXPORT_PATH, GCS_FILE_PATH, PROJECT_CRS,
SEGMENT_BUFFER_METERS, AM_PEAK, PM_PEAK, HQ_TRANSIT_THRESHOLD, MS_TRANSIT_THRESHOLD)

In [4]:
import sjoin_stops_to_segments

In [91]:
import create_aggregate_stop_frequencies

In [120]:
import importlib
importlib.reload(create_aggregate_stop_frequencies)

<module 'create_aggregate_stop_frequencies' from '/home/jovyan/data-analyses/high_quality_transit_areas/create_aggregate_stop_frequencies.py'>

In [121]:
analysis_date

'2024-10-21'

In [122]:
stop_times = helpers.import_scheduled_stop_times(
    analysis_date,
    get_pandas = True,
)

In [123]:
stop_times = create_aggregate_stop_frequencies.add_route_dir(stop_times, analysis_date)

In [105]:
# stop_times >> head(2)

In [124]:
st_prepped = stop_times.pipe(create_aggregate_stop_frequencies.prep_stop_times)

## multi logic

In [127]:
multi_test2 = create_aggregate_stop_frequencies.stop_times_aggregation_max_by_stop(st_prepped, analysis_date, single_route_dir=False)

## single logic

In [129]:
single_test2 = create_aggregate_stop_frequencies.stop_times_aggregation_max_by_stop(st_prepped, analysis_date, single_route_dir=True)

## create count of shared stops between each route_dir

In [126]:
min_freq = min([HQ_TRANSIT_THRESHOLD, MS_TRANSIT_THRESHOLD])

In [146]:
def get_explode_multiroute_only(
    single_route_aggregation: pd.DataFrame,
    multi_route_aggregation: pd.DataFrame,
    min_freqency: int
) -> pd.DataFrame:
    '''
    Shrink the problem space for the compute-intensive collinearity screen.
    First, get stops with any chance of qualifying as a major stop/hq corr for
    both single and multi-route aggregations.
    Then get stops that appear in multi-route qualifiers only, these will go to
    further processing.
    '''
    single_qual = single_route_aggregation >> filter(_.am_max_trips_hr > min_freqency, _.pm_max_trips_hr > min_freqency)
    multi_qual = multi_route_aggregation >> filter(_.am_max_trips_hr > min_freqency, _.pm_max_trips_hr > min_freqency)
    multi_only = multi_qual >> anti_join(_, single_qual, on=['schedule_gtfs_dataset_key', 'stop_id'])
    print(f'{multi_only.shape[0]} stops may qualify with multi-route aggregation')
    multi_only_explode = (multi_only[['schedule_gtfs_dataset_key', 'stop_id', 'route_dir']]
    .explode('route_dir')
    .sort_values(['schedule_gtfs_dataset_key','stop_id', 'route_dir'])) #  sorting crucial for next step
    return multi_only_explode

In [147]:
multi_only_explode = get_explode_multiroute_only(single_test2, multi_test2, min([HQ_TRANSIT_THRESHOLD, MS_TRANSIT_THRESHOLD]))

4560 stops may qualify with multi-route aggregation


In [261]:
multi_only_explode >> head(3)

,schedule_gtfs_dataset_key,stop_id,route_dir
96,0139b1253130b33adcd4b3a4490530d2,52c2636c-34a3-434c-99ae-cdf3dc36d15c,0e85fd4c-5258-4256-9852-4a96554aadb7_0
96,0139b1253130b33adcd4b3a4490530d2,52c2636c-34a3-434c-99ae-cdf3dc36d15c,T11x_0
96,0139b1253130b33adcd4b3a4490530d2,52c2636c-34a3-434c-99ae-cdf3dc36d15c,T1_0


In [205]:
def accumulate_share_count(route_dir_exploded: pd.DataFrame):
    '''
    For use via pd.DataFrame.groupby.apply
    Accumulate the number of times each route_dir shares stops with
    each other in a dictionary (share_counts)
    '''
    global share_counts
    rt_dir = route_dir_exploded.route_dir.to_numpy()
    schedule_gtfs_dataset_key = route_dir_exploded.schedule_gtfs_dataset_key.iloc[0]
    for route_dir in rt_dir:
        route = route_dir.split('_')[0] #  don't compare opposite dirs of same route, leads to edge cases like AC Transit 45
        other_dirs = [x for x in rt_dir if x != route_dir and x.split('_')[0] != route]
        for other_dir in other_dirs:
            key = schedule_gtfs_dataset_key+'__'+route_dir+'__'+other_dir
            if key in share_counts.keys():
                share_counts[key] += 1
            else:
                share_counts[key] = 1

In [206]:
share_counts = {}
multi_only_explode.groupby(['schedule_gtfs_dataset_key', 'stop_id']).apply(accumulate_share_count)

""


In [207]:
# share_counts

### Which threshold?

* 8 catches Muni 48 and 66, which are somewhat marginal but not an edge case per se

In [209]:
# qualify

## lookup function/filtering steps

1. If a feed has no route direction pairs qualifying, by definition no stops will qualify. Can exclude feed from next steps.
1. Get a list of unique feeds where at least one route direction pair qualifies to evaluate.

edge cases:

[AC Transit 45](https://www.actransit.org/sites/default/files/timetable_files/45-2023_12_03.pdf) _Opposite directions share a same-direction loop._ __Solved__ by preventing the same route from being compared with itself in the opposite direction.

[SDMTS 944/945](https://www.sdmts.com/sites/default/files/routes/pdf/944.pdf) _Shared frequent stops are few, and these routes are isolated._ __Solved__ by once again applying the `SHARED_STOP_THRESHOLD` after aggregation (by ensuring at least one route_dir at each stop has >= `SHARED_STOP_THRESHOLD` frequent stops). Complex typology including a loop route, each pair of [944, 945, 945A(946)] has >= threshold... but not actually in the same spots!

In [208]:
SHARED_STOP_THRESHOLD = 8
qualify = {key: share_counts[key] for key in share_counts.keys() if share_counts[key] >= SHARED_STOP_THRESHOLD}

In [216]:
feeds_to_filter = np.unique([key.split('__')[0] for key in qualify.keys()])

In [217]:
feeds_no_qualify = np.unique([key.split('__')[0] for key in share_counts.keys() if key.split('__')[0] not in feeds_to_filter])

In [218]:
from calitp_data_analysis.tables import tbls

In [219]:
feeds_no_qualify = tbls.mart_transit_database.dim_gtfs_service_data() >> filter(_.gtfs_dataset_key.isin(feeds_no_qualify)) >> distinct(_.name, _.gtfs_dataset_key) >> collect()

In [225]:
# feed_names = (tbls.mart_transit_database.dim_gtfs_service_data() >> filter(_.gtfs_dataset_key.isin(feeds_to_filter))
#  >> distinct(_.name, _.gtfs_dataset_key)
#  >> collect()
# )

In [365]:
feed_names_filtered = feed_names >> filter(_.name.str.contains('Long'))
display(feed_names_filtered)
gtfs_dataset_key = feed_names_filtered.gtfs_dataset_key.iloc[0]

,name,gtfs_dataset_key
23,Long Beach Transit – Long Beach Schedule,f1b35a50955aeb498533c1c6fdafbe44


In [227]:
# dataset_key = '015d67d5b75b5cf2b710bbadadfb75f5' #  Marin
# dataset_key = '3c62ad6ee589d56eca915ce291a5df0a' #  Yolobus 42A and 42B share 5+ stops so they match, which isn't desirable.
# dataset_key = '70c8a8b71c815224299523bf2115924a' #  SacRT
# dataset_key = '63029a23cb0e73f2a5d98a345c5e2e40' #  Elk Grove
# dataset_key = 'f1b35a50955aeb498533c1c6fdafbe44' #  LBT

In [354]:
def feed_level_filter(
gtfs_dataset_key: str,
multi_only_explode: pd.DataFrame,
qualify_dict: dict,
st_prepped: pd.DataFrame
) -> pd.DataFrame:
    '''
    For a single feed, filter potential stop_times to evaluate based on if their route_dir
    appears at all in qualifying route_dir dict, recheck if there's any chance those stops
    could qualify. Further shrinks problem space for check_stop lookup step
    '''

    this_feed_qual = {key.split(gtfs_dataset_key)[1][2:]:qualify_dict[key] for key in qualify_dict.keys() if key.split('__')[0] == gtfs_dataset_key}
    qualify_pairs = [tuple(key.split('__')) for key in this_feed_qual.keys()]
    arr = np.array(qualify_pairs[0])
    for pair in qualify_pairs[1:]: arr = np.append(arr, np.array(pair))
    any_appearance = np.unique(arr)

    #  only need to check stops that qualify as multi-route only
    stops_to_eval = multi_only >> filter(_.schedule_gtfs_dataset_key == gtfs_dataset_key) >> distinct(_.stop_id)
    st_prepped = st_prepped >> filter(_.schedule_gtfs_dataset_key == gtfs_dataset_key,
                                      _.stop_id.isin(stops_to_eval.stop_id),
                                     )
    print(f'{st_prepped.shape}')
    st_to_eval = st_prepped >> filter(_.route_dir.isin(any_appearance))
    print(f'{st_to_eval.shape}')
    #  cut down problem space by checking if stops still could qual after filtering for any appearance
    min_rows = min_freq * (len(am_peak_hrs) + len(pm_peak_hrs))
    st_could_qual = (st_to_eval >> group_by(_.stop_id)
     >> mutate(could_qualify = _.shape[0] >= min_rows)
     >> ungroup()
     >> filter(_.could_qualify)
    )
    print(f'{st_could_qual.shape}')
    return st_could_qual, qualify_pairs

In [355]:
st_could_qual, qualify_pairs = feed_level_filter(gtfs_dataset_key, multi_only_explode, qualify, st_prepped)

(9387, 16)
(4080, 16)
(4041, 17)


In [283]:
def check_stop(this_stop_route_dirs, qualify_pairs):
    #  check if all possible combinations included
    this_stop_route_dirs = list(this_stop_route_dirs)
    if len(this_stop_route_dirs) == 1:
        print('exhausted!')
        return []
    print(f'attempting {this_stop_route_dirs}... ', end='')
    stop_route_dir_pairs = list(itertools.combinations(this_stop_route_dirs, 2))
    checks = np.array([True if rt_dir in qualify_pairs else False for rt_dir in stop_route_dir_pairs])
    if checks.all():
        print(f'matched!')
        return this_stop_route_dirs
    else:
        print('subsetting...')
        this_stop_route_dirs.pop(-1)
        return check_stop(this_stop_route_dirs, qualify_pairs)

In [284]:
# check_stop(['no', 'nyet', 'bazz', 'fizz', 'buzz'], qualify_pairs)

In [285]:
def filter_qualifying_stops(one_stop_df, qualify_pairs):

    one_stop_df = (one_stop_df >> group_by(_.route_dir)
                >> mutate(route_dir_count = _.shape[0]) >> ungroup()
                >> arrange(-_.route_dir_count)
               )
    this_stop_route_dirs = (one_stop_df >> distinct(_.route_dir, _.route_dir_count)).route_dir.to_numpy() #  preserves sort order
    aggregation_ok_route_dirs = check_stop(this_stop_route_dirs, qualify_pairs)
    return one_stop_df >> filter(_.route_dir.isin(aggregation_ok_route_dirs))

In [303]:
df2 = st_could_qual.groupby('stop_id').apply(filter_qualifying_stops, qualify_pairs=qualify_pairs)

attempting ['31_1', '18_1']... matched!
attempting ['31_1', '18_1']... matched!
attempting ['31_1', '18_1']... matched!
attempting ['31_1', '18_1']... matched!
attempting ['9_1', '9R_1']... matched!
exhausted!
exhausted!
exhausted!
exhausted!
exhausted!
exhausted!
exhausted!
exhausted!
attempting ['31_1', '18_1']... matched!
attempting ['31_1', '18_1']... matched!
attempting ['48_1', '18_1']... subsetting...
exhausted!
attempting ['31_1', '18_1']... matched!
attempting ['5R_1', '5R_0', '31_1', '5_0', '5_1']... subsetting...
attempting ['5R_1', '5R_0', '31_1', '5_0']... subsetting...
attempting ['5R_1', '5R_0', '31_1']... subsetting...
attempting ['5R_1', '5R_0']... subsetting...
exhausted!
exhausted!
attempting ['5R_0', '5_0']... matched!
attempting ['5R_1', '5_1']... matched!
attempting ['5R_0', '5_0']... matched!
attempting ['5R_1', '5_1']... matched!
attempting ['5R_1', '5_1']... matched!
attempting ['5R_0', '5_0']... matched!
attempting ['5R_1', '5_1']... matched!
attempting ['5R_1

In [304]:
df2 = df2.reset_index(drop=True)

In [305]:
df2.stop_id.unique()

array(['13059', '13062', '13064', '13066', '13244', '13570', '13572',
       '13925', '14211', '14212', '14213', '14215', '14218', '14221',
       '14222', '14228', '14231', '14232', '14732', '14734', '14736',
       '14737', '14742', '14746', '14747', '14748', '14749', '14751',
       '14753', '14809', '14842', '14843', '14895', '15658', '15685',
       '16047', '16049', '16051', '16058', '16059', '16063', '16068',
       '16069', '16072', '16073', '16075', '16139', '16141', '16143',
       '16145', '16147', '16149', '16151', '16153', '16339', '16344',
       '16383', '16385', '16386', '16388', '16565', '16568', '16570',
       '16575', '16584', '17204', '17264', '17297', '17720', '17721',
       '17727', '17728', '17729', '17730'], dtype=object)

In [307]:
trips_per_peak_multi = create_aggregate_stop_frequencies.stop_times_aggregation_max_by_stop(df2, analysis_date, single_route_dir=False)

In [318]:
trips_per_peak_multi_filtered = trips_per_peak_multi >> filter(_.am_max_trips_hr > min_freq, _.pm_max_trips_hr > min_freq)

In [319]:
short_routes = trips_per_peak_multi_filtered.explode('route_dir') >> count(_.route_dir) >> filter(_.n < SHARED_STOP_THRESHOLD)

In [320]:
short_routes

,route_dir,n


In [326]:
df2_final['all_short'] = df2_final.route_dir.map(lambda x: np.array([True if y in list(short_routes.route_dir) else False for y in x]).all())

In [327]:
df2_final >> filter(_.all_short) #  stops where _every_ shared route has less than SHARED_STOP_THRESHOLD frequent stops (even after aggregation)

,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short


In [328]:
df2_final = df2_final >> filter(-_.all_short)

## unify function, try looping over all feeds?

In [360]:
def collinear_filter_feed(
    gtfs_dataset_key: str,
    multi_only_explode: pd.DataFrame,
    qualify_dict: dict,
    st_prepped: pd.DataFrame
):
    
    st_could_qual, qualify_pairs = feed_level_filter(gtfs_dataset_key, multi_only_explode, qualify, st_prepped)
    st_qual_filter_1 = st_could_qual.groupby('stop_id').apply(filter_qualifying_stops, qualify_pairs=qualify_pairs)
    st_qual_filter_1 = st_qual_filter_1.reset_index(drop=True)
    if st_qual_filter_1.empty: return
    feed_key = st_qual_filter_1.feed_key.iloc[0]
    trips_per_peak_qual_1 = create_aggregate_stop_frequencies.stop_times_aggregation_max_by_stop(st_qual_filter_1, analysis_date, single_route_dir=False)
    trips_per_peak_qual_2 = trips_per_peak_multi >> filter(_.am_max_trips_hr > min_freq, _.pm_max_trips_hr > min_freq)
    short_routes = trips_per_peak_qual_2.explode('route_dir') >> count(_.route_dir) >> filter(_.n < SHARED_STOP_THRESHOLD)
    print('short routes, all_short stops:')
    display(short_routes)
    trips_per_peak_qual_2['all_short'] = trips_per_peak_qual_2.route_dir.map(
        lambda x: np.array([True if y in list(short_routes.route_dir) else False for y in x]).all())
    display(trips_per_peak_qual_2 >> filter(_.all_short)) #  stops where _every_ shared route has less than SHARED_STOP_THRESHOLD frequent stops (even after aggregation)
    trips_per_peak_qual_3 = trips_per_peak_qual_2 >> filter(-_.all_short)
    trips_per_peak_qual_3['feed_key'] = feed_key #  for mapping in dev, can get rid of
    
    return trips_per_peak_qual_3

In [361]:
# muni_final = collinear_filter_feed(dataset_key, multi_only_explode, qualify, st_prepped)

In [366]:
lbt = collinear_filter_feed(gtfs_dataset_key, multi_only_explode, qualify, st_prepped)

(15120, 16)
(13338, 16)
(13005, 17)
attempting ['171_1', '172_1', '173_1', '175_1']... matched!
attempting ['191_1', '192_1']... matched!
attempting ['191_1', '192_1']... matched!
attempting ['191_1', '192_1']... matched!
attempting ['191_1', '192_1']... matched!
attempting ['191_1', '192_1']... matched!
attempting ['191_1', '192_1']... matched!
attempting ['41_0', '191_1', '192_1']... subsetting...
attempting ['41_0', '191_1']... subsetting...
exhausted!
attempting ['41_0', '191_1', '192_1']... subsetting...
attempting ['41_0', '191_1']... subsetting...
exhausted!
attempting ['41_0', '191_1', '192_1']... subsetting...
attempting ['41_0', '191_1']... subsetting...
exhausted!
attempting ['41_0', '191_1', '192_1']... subsetting...
attempting ['41_0', '191_1']... subsetting...
exhausted!
attempting ['191_1', '192_1']... matched!
attempting ['191_1', '192_1']... matched!
attempting ['191_1', '192_1']... matched!
attempting ['191_1', '192_1']... matched!
attempting ['191_1', '192_1']... mat

,route_dir,n


,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short


In [362]:
all_collinear = pd.DataFrame()
for gtfs_dataset_key in feeds_to_filter:
    df = collinear_filter_feed(gtfs_dataset_key, multi_only_explode, qualify, st_prepped)
    all_collinear = pd.concat([df, all_collinear])

(1818, 16)
(865, 16)
(664, 17)
attempting ['22_1', '23_1', '68_1']... matched!
attempting ['22_1', '23_1', '68_1']... matched!
attempting ['22_1', '23_1', '68_1']... matched!
attempting ['22_1', '23_1', '68_1']... matched!
attempting ['22_1', '23_1', '68_1']... matched!
attempting ['22_1', '23_1', '68_1']... matched!
attempting ['35_0', '36_1', '23_1']... matched!
attempting ['35_0', '36_1', '23_1']... matched!
attempting ['35_0', '36_1', '23_1']... matched!
attempting ['35_0', '36_1', '23_1']... matched!
attempting ['35_0', '36_1', '23_1']... matched!
attempting ['35_0', '36_1', '23_1']... matched!
attempting ['35_0', '36_1', '23_1']... matched!
attempting ['35_0', '36_1', '23_1']... matched!
attempting ['22_1', '23_1', '68_1']... matched!
attempting ['22_1', '23_1', '68_1']... matched!
short routes, all_short stops:


,route_dir,n


,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short


(23191, 16)
(10061, 16)
(9019, 17)
attempting ['94-13183_1', '92-13183_1']... subsetting...
exhausted!
exhausted!
attempting ['165-13183_1', '294-13183_0']... matched!
attempting ['165-13183_0', '294-13183_1']... matched!
attempting ['165-13183_0', '294-13183_1']... matched!
attempting ['165-13183_0', '294-13183_1']... matched!
exhausted!
exhausted!
attempting ['92-13183_1', '222-13183_1']... matched!
attempting ['92-13183_1', '222-13183_1']... matched!
attempting ['92-13183_1', '222-13183_1']... matched!
attempting ['92-13183_0', '222-13183_0']... matched!
attempting ['92-13183_1', '222-13183_1']... matched!
attempting ['92-13183_1', '222-13183_1']... matched!
attempting ['92-13183_1', '222-13183_1']... matched!
attempting ['294-13183_1', '222-13183_1']... subsetting...
exhausted!
attempting ['117-13183_1', '120-13183_1']... matched!
attempting ['117-13183_1', '120-13183_1']... matched!
attempting ['117-13183_1', '120-13183_1']... matched!
attempting ['117-13183_1', '120-13183_1']... 

,route_dir,n


...

,route_dir,n


,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short


(13206, 16)
(5149, 16)
(4982, 17)
attempting ['22_0', '77_1']... matched!
attempting ['22_0', '77_1']... matched!
attempting ['77_1', '22_0']... matched!
attempting ['77_1', '22_0']... matched!
attempting ['77_1', '22_0']... matched!
attempting ['77_1', '22_0']... matched!
attempting ['77_1', '22_0']... matched!
attempting ['77_1', '22_0']... matched!
attempting ['77_1', '22_0']... matched!
exhausted!
exhausted!
exhausted!
exhausted!
exhausted!
exhausted!
attempting ['68_0', 'Rapid 568_0']... matched!
attempting ['68_0', 'Rapid 568_0']... matched!
exhausted!
exhausted!
exhausted!
exhausted!
exhausted!
exhausted!
exhausted!
exhausted!
exhausted!
exhausted!
exhausted!
exhausted!
exhausted!
exhausted!
exhausted!
attempting ['22_0', '66_0']... subsetting...
exhausted!
exhausted!
exhausted!
exhausted!
exhausted!
exhausted!
exhausted!
attempting ['22_1', '64B_1']... subsetting...
exhausted!
attempting ['22_1', '64B_1']... subsetting...
exhausted!
attempting ['22_1', 'Rapid 522_1']... matched

,route_dir,n


,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short


## Map single result

In [337]:
stops = helpers.import_scheduled_stops(
    analysis_date,
    get_pandas = True,
    crs = PROJECT_CRS
)

In [369]:
lbt

,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short,feed_key
0,7cc0cb1871dfd558f11a2885c145d144,13059,17,"[18_1, 31_1]",24,5,6,41,2,False,18dc7c803a632739be475747f212df94
1,7cc0cb1871dfd558f11a2885c145d144,13062,17,"[18_1, 31_1]",24,5,6,41,2,False,18dc7c803a632739be475747f212df94
2,7cc0cb1871dfd558f11a2885c145d144,13064,17,"[18_1, 31_1]",24,5,6,41,2,False,18dc7c803a632739be475747f212df94
3,7cc0cb1871dfd558f11a2885c145d144,13066,17,"[18_1, 31_1]",24,5,6,41,2,False,18dc7c803a632739be475747f212df94
4,7cc0cb1871dfd558f11a2885c145d144,13244,17,"[9R_1, 9_1]",39,5,9,56,2,False,18dc7c803a632739be475747f212df94
...,...,...,...,...,...,...,...,...,...,...,...
69,7cc0cb1871dfd558f11a2885c145d144,17721,15,"[5R_1, 5_1]",20,5,5,35,2,False,18dc7c803a632739be475747f212df94
70,7cc0cb1871dfd558f11a2885c145d144,17727,16,"[5R_0, 5_0]",19,5,4,35,2,False,18dc7c803a632739be475747f212df94
71,7cc0cb1871dfd558f11a2885c145d144,17728,16,"[5R_0, 5_0]",20,5,5,36,2,False,18dc7c803a632739be475747f212df94
72,7cc0cb1871dfd558f11a2885c145d144,17729,15,"[5R_0, 5_0]",20,5,5,35,2,False,18dc7c803a632739be475747f212df94


In [367]:
gdf = stops >> inner_join(_, lbt, on = ['feed_key', 'stop_id']) >> distinct(_.stop_id, _.geometry)

In [368]:
gdf.explore()

ValueError: Location values cannot contain NaNs.

In [302]:
df2_final >> filter(_.stop_id == '40010')

,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short


## Map overall results

In [211]:
gdf = stops >> inner_join(_, multi_only, on = ['stop_id']) >> select(_.stop_id, _.geometry)

In [363]:
gdf2 = stops >> inner_join(_, all_collinear, on = ['feed_key', 'stop_id']) >> select(_.stop_id, _.geometry)

In [213]:
gdf2.geometry = gdf2.buffer(400)

In [214]:
gdf = gdf.overlay(gdf2, how='difference')

In [215]:
# gdf.explore()

In [370]:
# gdf2.explore()